In [ ]:
import cv2
import numpy as np
import pywt

Total Detections: 1401


In [ ]:
dataset_path = "videos_of_wildfires/Dramatic_evacuations_by_sea_as_forest_fires_rage_in_Italy_and_Turkey.mp4"

In [ ]:
if dataset_path.endswith((".mp4", ".avi", ".mov")):
    cap = cv2.VideoCapture(dataset_path)

    prev_frame = None
    detection_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        lower_fire_hsv = np.array([5, 150, 150])  
        upper_fire_hsv = np.array([25, 255, 255])  
        fire_mask_hsv = cv2.inRange(hsv, lower_fire_hsv, upper_fire_hsv)

        a_channel = lab[:, :, 1]
        _, fire_mask_lab = cv2.threshold(a_channel, 160, 255, cv2.THRESH_BINARY)

        red_channel = frame[:, :, 2]
        _, fire_mask_rgb = cv2.threshold(red_channel, 190, 255, cv2.THRESH_BINARY)

        combined_fire_mask = cv2.bitwise_or(cv2.bitwise_or(fire_mask_hsv, fire_mask_rgb), fire_mask_lab)

        kernel = np.ones((3, 3), np.uint8)
        combined_fire_mask = cv2.morphologyEx(combined_fire_mask, cv2.MORPH_ERODE, kernel)

        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        normalized_gray = clahe.apply(gray)

        blur1 = cv2.GaussianBlur(normalized_gray, (5, 5), 1)
        blur2 = cv2.GaussianBlur(normalized_gray, (11, 11), 2)
        dog = cv2.absdiff(blur1, blur2)

        coeffs2 = pywt.dwt2(dog, 'haar')
        LL, (LH, HL, HH) = coeffs2
        wavelet_enhanced = cv2.convertScaleAbs(HH)

        fire_detected = False
        fire_contours, _ = cv2.findContours(combined_fire_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for cnt in fire_contours:
            area = cv2.contourArea(cnt)

            if area > 250:  
                x, y, w, h = cv2.boundingRect(cnt)

                region = gray[y:y+h, x:x+w]
                edges = cv2.Canny(region, 50, 150)
                edge_density = np.sum(edges) / (w * h)

                if edge_density < 0.02:  
                    continue

                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                fire_detected = True

        if prev_frame is not None:
            diff = cv2.absdiff(prev_frame, gray)
            _, flicker_mask = cv2.threshold(diff, 20, 255, cv2.THRESH_BINARY)

            flicker_pixels = np.count_nonzero(flicker_mask)

            if fire_detected and flicker_pixels < 1000:  
                fire_detected = False

        prev_frame = gray.copy()

        if fire_detected:
            detection_count += 1
            cv2.putText(frame, "WILDFIRE DETECTED!", (50, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)

        cv2.imshow("Fire Mask", combined_fire_mask)
        cv2.imshow("Frame", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

print(f"Total Detections: {detection_count}")
